## Name: 

In [8]:
from copy import deepcopy
import random
import numpy as np

from cache import Cache, CacheAssoc

## Matrix Multiplication and Memory Access Pattern

We will cheeck the memory access pattern for matrix multiplication. Consider this matrix multiplication:

$$
C_{m,n} = \sum_k A_{m,k} \times B_{k,n}
$$

Suppose our processor doesn't have a cache and directly loads operands from the main memory and stores the results back to the main memory. We want to count how many memory accesses are necessary to complete the computation, and estimate how much energy will be consumed for the memory access. 

In [9]:
# Initialize values
seed = 10
random.seed(seed)

M = 4
K = 4
N = 4

# Matrix A (M x N)
a_MK = []
for m in range(M):
    a_MK.append([random.randint(1, 9) for i in range(K)])


# Matrix B (K x N)
b_KN = []
for k in range(K):
    b_KN.append([random.randint(1, 9) for i in range(N)])
    
# print(a_MK)
# print(b_KN)

# ground truth for the result (M x N)
c_MN = []
for m in range(M):
    temp = []
    for n in range(N):
        t = 0
        for k in range(K):
            t += a_MK[m][k] * b_KN[k][n]
        temp.append(t)
    c_MN.append(temp)
# print(c_MN)

### Question 1

How many memory reads are in this loop nest? Break down the number of memory reads for each matrix (A, B, and C).

How many memory writes are in this loop nest? Assume there is no register for storing partial sums, and reading and writing intermediate results should also be considered. Break down the number of memory writes for each matrix (A, B, and C).

Fill in your answers in the table below:

|   | A | B | C |
| --- | --- | --- | --- |
| Read | 64 | 64 | 64 |
| Write | 0 | 0 | 64 |


In [10]:
# Loop nest for the matrix multiplication
c = np.zeros((M, N))
for m in range(M):
    for n in range(N):
        for k in range(K):
            # load a
            a = a_MK[m][k]
            
            # load b
            b = b_KN[k][n]
            
            # compute partial sum
            c[m][n] += a * b

# Check the result
print(np.all(c==np.asarray(c_MN)))

True


## Direct Mapped Cache

Suppose our processor uses a directed mapped cache. We want to count the number of cache hit and miss for both read and write operations. Assume all data are initially stored in the main memory in __row major__ order. 

We are using two parameters when defining a cache: `log_size` and `words_per_line`. The total number of __words__ that a cache can store is $2^{log\_size}$, and the number of words per line (block size) is `words_per_line`. For example, if a cache has `log_size=4`, and `words_per_line=2`, this cache can store total 16 words with block size of 2. We assume that our direct mapped cache uses write-through with no write allocation for store operations. 

In [11]:
def read_addr(addr, memory, cache):
    val = memory[addr]
    cache.load(addr)
    return val
    
def write_addr(addr, memory, cache, val=0):
    memory[addr] = val
    cache.store(addr)

### Question 2

Modify the address that the processor has to request for read and write operation in the loop nest.

In [21]:
# Define a main memory
mem_log_size = 10
memory = np.zeros(2**mem_log_size).astype(np.int32)
memory[:M*K] = np.asarray(a_MK).flatten()
memory[M*K:M*K+K*N] = np.asarray(b_KN).flatten()

# Define a cache for each matrix (A, B, C)
cache_a = Cache(log_size=3, words_per_line=1)
cache_b = Cache(log_size=3, words_per_line=1)
cache_c = Cache(log_size=3, words_per_line=1)

# Loop nest for matrix multiplication
for m in range(M):
    for n in range(N):
        for k in range(K):

            # Your code: modify addr_a 
            addr_a = m*K + k + 0
            a = read_addr(addr_a, memory, cache_a)
            
            # Your code: modify addr_b
            addr_b = k*N + n + M*K
            b = read_addr(addr_b, memory, cache_b)
            
            # Your code: modify addr_c
            addr_c = m*N + n + M*K + K*N
            psum = read_addr(addr_c, memory, cache_c)
            
            # compute partial sum
            psum += a * b
            write_addr(addr_c, memory, cache_c, int(psum))

# Check the result
print(np.all(memory[M*K+K*N:M*K+K*N+M*N].reshape((M, N))==np.asarray(c_MN)))

# Print cache statistics
print("-------Cache A--------")
cache_a.print_stats()
print("-------Cache B--------")
cache_b.print_stats()
print("-------Cache C--------")
cache_c.print_stats()

True
-------Cache A--------
Cache Statistics:
cache rd: 48
cache wr: 0
mem rd: 16
mem wr: 0
-------Cache B--------
Cache Statistics:
cache rd: 0
cache wr: 0
mem rd: 64
mem wr: 0
-------Cache C--------
Cache Statistics:
cache rd: 48
cache wr: 64
mem rd: 16
mem wr: 64


### Question 3

Now, run the same loop nest using caches with the same size, but with different words per line (block size). Explain why the statistics for Cache A is different from the previous case.

The cache with >1 word per line can exploit **data locality** to reduce cache misses, because our multiplication algorithm performs concordant traversal of the input matrices (in terms which apply to dense matrices, this means that we are iterating through elements of the row-major matrix representation in consecutive order.) A consequence of concordant traversal is that if we have just accessed A[i]  (i and j here are flattened positions), then it is very likely that we will subsequently access A[i+1]. So when the first element A[i] of a row is read, for example, the cache line will hold not just A[i] but also {A[i+d] for d <= cache line words}. Then when the concordant traversal accesses the next element A[i+1] in the row, it is a cache hit. Effectively, long cache lines increase cache hits by anticipating that programs will implement local, consecutive memory accesses.

However, B cannot reap the benefits of the long cache lines, because we are not performing a concordant traversal of B - the inner-most loop of the multiplication algorithm jumps from row to row of B, and is therefore effectively making strided jumps through the flattened row-major represention of B. Therefore, the assumption built into the long cache-line optimization - that data access is local and consecutive - is incorrect for B, and so there are no cache hits.

In [22]:
# Define a main memory
mem_log_size = 10
memory = np.zeros(2**mem_log_size).astype(np.int32)
memory[:M*K] = np.asarray(a_MK).flatten()
memory[M*K:M*K+K*N] = np.asarray(b_KN).flatten()

# Define a cache for each matrix (A, B, C)
cache_a = Cache(log_size=3, words_per_line=2)
cache_b = Cache(log_size=3, words_per_line=2)
cache_c = Cache(log_size=3, words_per_line=2)

# Loop nest for matrix multiplication
for m in range(M):
    for n in range(N):
        for k in range(K):

            # Your code: modify addr_a 
            addr_a = m*K + k + 0
            a = read_addr(addr_a, memory, cache_a)
            
            # Your code: modify addr_b
            addr_b = k*N + n + M*K
            b = read_addr(addr_b, memory, cache_b)
            
            # Your code: modify addr_c
            addr_c = m*N + n + M*K + K*N
            psum = read_addr(addr_c, memory, cache_c)
            
            # compute partial sum
            psum += a * b
            write_addr(addr_c, memory, cache_c, int(psum))

# Check the result
print(np.all(memory[M*K+K*N:M*K+K*N+M*N].reshape((M, N))==np.asarray(c_MN)))

# Print cache statistics
print("-------Cache A--------")
cache_a.print_stats()
print("-------Cache B--------")
cache_b.print_stats()
print("-------Cache C--------")
cache_c.print_stats()

True
-------Cache A--------
Cache Statistics:
cache rd: 56
cache wr: 0
mem rd: 8
mem wr: 0
-------Cache B--------
Cache Statistics:
cache rd: 0
cache wr: 0
mem rd: 64
mem wr: 0
-------Cache C--------
Cache Statistics:
cache rd: 56
cache wr: 64
mem rd: 8
mem wr: 64


### Question 4

Suppose we store data of Matrix B to the main memory in __column major__ order. Change the address that are requested by the processor, and explain the difference of the statistics for Cache B.


The cache statistics demonstrate that 50% of B reads are now cache hits. Our multiplication algorithm naturally iterates through B in column-major order (the inner loop iterates over rank k which corresponds to columns of B); by adopting a column-major representation, the iteration because concordant and so there is significant data locality. If B[i] is accessed, it is very likely B[i+1] will be accessed next, and so the long cache lines can hold chunks of words from a column of B in anticipation of local consecutive accesses. This increases the number of cache hits.



In [24]:
mem_log_size = 10
memory_col = np.zeros(2**mem_log_size).astype(np.int32)
memory_col[:M*K] = np.asarray(a_MK).flatten()
memory_col[M*K:M*K+K*N] = np.asarray(b_KN).transpose().flatten()

# Define a cache for each matrix (A, B, C)
cache_a = Cache(log_size=3, words_per_line=2)
cache_b = Cache(log_size=3, words_per_line=2)
cache_c = Cache(log_size=3, words_per_line=2)

# Loop nest for matrix multiplication
for m in range(M):
    for n in range(N):
        for k in range(K):

            # Your code: modify addr_a 
            addr_a = m*K + k + 0
            a = read_addr(addr_a, memory_col, cache_a)
            
            # Your code: modify addr_b
            addr_b = n*K + M*K + k 
            b = read_addr(addr_b, memory_col, cache_b)
            
            # Your code: modify addr_c
            addr_c = m*N + n + M*K + K*N
            psum = read_addr(addr_c, memory_col, cache_c)
            
            # compute partial sum
            psum += a * b
            write_addr(addr_c, memory_col, cache_c, int(psum))

# Check the result
print(np.all(memory_col[M*K+K*N:M*K+K*N+M*N].reshape((M, N))==np.asarray(c_MN)))

# Print cache statistics
print("-------Cache A--------")
cache_a.print_stats()
print("-------Cache B--------")
cache_b.print_stats()
print("-------Cache C--------")
cache_c.print_stats()

True
-------Cache A--------
Cache Statistics:
cache rd: 56
cache wr: 0
mem rd: 8
mem wr: 0
-------Cache B--------
Cache Statistics:
cache rd: 32
cache wr: 0
mem rd: 32
mem wr: 0
-------Cache C--------
Cache Statistics:
cache rd: 56
cache wr: 64
mem rd: 8
mem wr: 64


### Question 5

Suppose our loop nest order is different, from M -> N -> K to K -> N -> M. Assume all conditions are identical to Question 2. Explain cache statistics compared to those in Question 2. 

In [25]:
mem_log_size = 10
memory = np.zeros(2**mem_log_size).astype(np.int32)
memory[:M*K] = np.asarray(a_MK).flatten()
memory[M*K:M*K+K*N] = np.asarray(b_KN).flatten()

# Define a cache for each matrix (A, B, C)
cache_a = Cache(log_size=3, words_per_line=1)
cache_b = Cache(log_size=3, words_per_line=1)
cache_c = Cache(log_size=3, words_per_line=1)

for k in range(K):
    for n in range(N):
        for m in range(M):
            # Your code: modify addr_a 
            addr_a = m*K + k + 0
            a = read_addr(addr_a, memory, cache_a)
            
            # Your code: modify addr_a 
            addr_b = k*N + n + M*K
            b = read_addr(addr_b, memory, cache_b)
            
            # Your code: modify addr_a 
            addr_c = m*N + n + M*K + K*N
            psum = read_addr(addr_c, memory, cache_c)

            # compute partial sum
            psum += a * b
            write_addr(addr_c, memory, cache_c, int(psum))


# Check the result
print(np.all(memory[M*K+K*N:M*K+K*N+M*N].reshape((M, N))==np.asarray(c_MN)))

# Print cache statistics
print("-------Cache A--------")
cache_a.print_stats()
print("-------Cache B--------")
cache_b.print_stats()
print("-------Cache C--------")
cache_c.print_stats()

True
-------Cache A--------
Cache Statistics:
cache rd: 0
cache wr: 0
mem rd: 64
mem wr: 0
-------Cache B--------
Cache Statistics:
cache rd: 48
cache wr: 0
mem rd: 16
mem wr: 0
-------Cache C--------
Cache Statistics:
cache rd: 0
cache wr: 64
mem rd: 64
mem wr: 64


## Set Associative Cache

Now, we will consider a set associative cache. This cache has another parameter, `num_ways`, that specifies the number of ways. For example, if `log_size=4`, `words_per_line=2`, and `num_ways=2`, this cache can store total 16 words and there are total 4 sets, where each set has two ways and the block size will be two. Similar to the direct mapped cache we discussed above, our set associative cache uses write-through with no write allocation for store, and uses Bit Psuedo Least Recently Used to replace cache entries. 

### Question 6

Let's run the same loop nest as in Question 2, except for that we use one large cache instead of using three different caches for each matrix. Note that we are still using a direct mapped cache here. Explain the difference in the total cache hit and miss ratio, compared to Question 2. 


_your-answer-here_



In [16]:
# Define a main memory
mem_log_size = 10
memory = np.zeros(2**mem_log_size).astype(np.int32)
memory[:M*K] = np.asarray(a_MK).flatten()
memory[M*K:M*K+K*N] = np.asarray(b_KN).flatten()

# Define a cache for each matrix (A, B, C)
cache = Cache(log_size=5, words_per_line=2)

# Loop nest for matrix multiplication
for m in range(M):
    for n in range(N):
        for k in range(K):

            # Your code: modify addr_a 
            addr_a = 0
            a = read_addr(addr_a, memory, cache)
            
            # Your code: modify addr_b
            addr_b = 0
            b = read_addr(addr_b, memory, cache)
            
            # Your code: modify addr_c
            addr_c = 0
            psum = read_addr(addr_c, memory, cache)
            
            # compute partial sum
            psum += a * b
            write_addr(addr_c, memory, cache, int(psum))
            
            # debug
            # print("---------------------")
            # print(addr_a, addr_b, addr_c)

# Check the result
print(np.all(memory[M*K+K*N:M*K+K*N+M*N].reshape((M, N))==np.asarray(c_MN)))

# Print cache statistics
print("-------Cache--------")
cache.print_stats()

False
-------Cache--------
Cache Statistics:
cache rd: 191
cache wr: 64
mem rd: 1
mem wr: 64


/tmp/ipykernel_230/1569185727.py:28: RuntimeWarning: overflow encountered in int_scalars
  psum += a * b


### Question 7

Observe the cache access pattern when we use a set associative cache with 2 ways. Explain the difference compared to Question 6. Generally, in which cases using set associative caches beneficial compared to direct mapped caches? If we use three separate caches for each matrix as before, do set associative caches provide benefits?


_your-answer-here_



In [17]:
# Define a main memory
mem_log_size = 10
memory = np.zeros(2**mem_log_size).astype(np.int32)
memory[:M*K] = np.asarray(a_MK).flatten()
memory[M*K:M*K+K*N] = np.asarray(b_KN).flatten()

# Define a cache for each matrix (A, B, C)
cache = CacheAssoc(log_size=5, words_per_line=2, num_ways=2)

# Loop nest for matrix multiplication
for m in range(M):
    for n in range(N):
        for k in range(K):

            # Your code: modify addr_a 
            addr_a = 0
            a = read_addr(addr_a, memory, cache)
            
            # Your code: modify addr_b
            addr_b = 0
            b = read_addr(addr_b, memory, cache)
            
            # Your code: modify addr_c
            addr_c = 0
            psum = read_addr(addr_c, memory, cache)
            
            # compute partial sum
            psum += a * b
            write_addr(addr_c, memory, cache, int(psum))
            
            # debug
            # print("---------------------")
            # print(addr_a, addr_b, addr_c)

# Check the result
print(np.all(memory[M*K+K*N:M*K+K*N+M*N].reshape((M, N))==np.asarray(c_MN)))

# Print cache statistics
print("-------Cache--------")
cache.print_stats()

False
-------Cache--------
Cache Statistics:
cache rd: 191
cache wr: 64
mem rd: 1
mem wr: 64


/tmp/ipykernel_230/4283516884.py:28: RuntimeWarning: overflow encountered in int_scalars
  psum += a * b
